<img align="right" width="200" height="200" src="https://static.tildacdn.com/tild6236-6337-4339-b337-313363643735/new_logo.png">

# Введение в Spark: RDD API
**Андрей Титов**  
tenke.iu8@gmail.com  

## На этом занятии
+ Общие сведения
+ Область применения
+ Архитектура приложений
+ Базовые функции RDD API
+ Рair RDD функции
+ Работа с данными

## Общие сведения

**Apache Spark** - это:
+ Платформа для построения распределнных приложений обработки данных
+ Эволюция Hadoop MapReduce
+ Библиотека под Python/Scala
+ Один из самых популярных проектов в области обработки больших данных


## Область применения
- Распределенная обработка больших данных
- Построение ETL пайплайнов
- Работа со структурированными данными (SQL)
- Разработка стриминг приложений

## Архитектура приложения

+ **Driver** (aka Master):
  - предоставляет API через SparkSession и SparkContext
  - выполняет ваш код - python файл или скомпилированный .jar
  - контролирует выполнение задачи

+ **Workers** (aka Executors or Slaves):
  - обрабатывают данные
  - каждый Worker работает со своим сегментом данных - **Partition**
  - не выполняются ваш код напрямую
  - получают задачи от Driver
+ **Cluster Manager** (YARN/Mesos):
  - отвечает за аллокацию контейнеров, выполняющих код драйвера и воркеров, на кластере
  - квотирует ресурсы между пользователями
  - контролирует состояние контейнеров
 
<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-06-05%2013.07.01.jpeg">

## Resilient Distributed Dataset

**RDD** aka Resilient Distributet Dataset - самая базовая и самая низкоуровневая структура в Spark, доступная разработчику. Представляет собой типизированную неизменяемую неупорядоченную партиционированную коллекцию данных, распределенную по узлам кластера

In [ ]:
val cities: Vector[String] = Vector("Moscow", "Paris", "Madrid", "London", "New York")
println(s"The Vector has ${cities.length} elements, the first one is ${cities.head}}")

In [ ]:
// val spark = SparkSession.builder.getOrCreate()
spark

In [ ]:
sc

In [ ]:
import org.apache.spark.rdd.RDD

val rdd = sc.parallelize(cities)
println(s"The RDD has ${rdd.count} elements, the first one is ${rdd.first}")

RDD может быть создана из:
- локальной коллекции на драйвере
- файла (локального или на распределенной файловой системе, например HDFS)
- базы данных

### Операции с RDD
1. Трансформации (e.g. map, filter)
2. Действия (e.g. reduce, collect, count, foreach)

**Трансформации** (Transormations):
- всегда превращают один RDD в новый RDD
- всегда являются ленивыми - создают граф вычислений, но не запускают их
- иногда (часто) неявно требуют перемешивания данных между воркерами - **Shuffle**

In [ ]:
// Трансформация map: не запускает вычислений, не изменяет изначальный RDD
val upperRdd: RDD[String] = rdd.map ( city => city.toUpperCase )


// Метод take возвращает N первых элементов RDD
val upperVec = upperRdd.take(3)
val oldVec = rdd.take(3)

// метод mkString позволяет сделать из любой локальной коллекции строку
println(s"""New RDD: ${upperVec.mkString(", ")}""")
println(s"""Old RDD: ${oldVec.mkString(", ")}""")

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2019.15.06.jpeg">

**Действия** (Actions):
- выполняют действие над RDD
- запускают вычисления

In [ ]:
// Действие reduce применяет функцию f к промежуточному результату 
// от предыдущей итерации со следующим элементом коллекции
val count = rdd.map( x => x.length ).reduce { (x,y) => x + y }
println(s"The RDD contains ${count} letters")

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2019.22.10.jpeg">

### Примеры операций с RDD

Фильтрация RDD

In [ ]:
val startsWithM = upperRdd.filter( x => x.startsWith("M") )
println(s"""The following city names starts with M: ${startsWithM.take(2).mkString(", ")}""")

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2019.33.22.jpeg">

Подсчет количества элементов в RDD

In [ ]:
val countM: Long = startsWithM.count()
println(s"The RDD contains $countM elements")

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2019.45.03.jpeg">

Передача ВСЕХ элементов RDD на драйвер

In [ ]:
val localArray: Array[String] = startsWithM.collect()
val containsMoscow: Boolean = localArray.contains("MOSCOW")
println(s"The array contains MOSCOW: $containsMoscow")

Передача N элементов по сети на драйвер

In [ ]:
val twoElements: Array[String] = startsWithM.take(2)
println(s"""Two elements of the RDD are: ${twoElements.mkString(", ")}""")

Сортировка и выборка из N первых элементов

In [ ]:
val twoElementsSorted: Array[String] = startsWithM.takeOrdered(2)
println(s"""Two elements of the RDD are: ${twoElementsSorted.mkString(", ")}""")

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2020.40.09.jpeg">

Спрямление вложенных коллекций

In [ ]:
val mappedRdd: RDD[Vector[Char]] = rdd.map(x => x.toVector)
mappedRdd.take(2).foreach(println)
println("####")

val flatMappedRdd = rdd.flatMap( x => x.toLowerCase )
flatMappedRdd.take(4).foreach(println)

Удаление дубликатов

In [ ]:
val uniqueLetters: String = 
    flatMappedRdd
        .distinct
        .filter(x => x != ' ')
        .collect
        .sorted
        .mkString(" ")

println(s"Letters in the RDD are: ${uniqueLetters}")

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2020.42.42.jpeg">

### Выводы
- RDD - это неизменяемый распределенный набор данных
- Трансформации (map, filter, flatMap) создают новый RDD из существующего и не изменяют существующий
- Любые трансформации являются ленивыми и не запускают вычислений
- Действия (count, reduce, collect, take) запускают вычисления

### Полезные ссылки:
- [RDD API Reference](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD)
- [RDD Programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
- [Scala 2.11.12 API](https://www.scala-lang.org/files/archive/api/2.11.12)

## PairRDD функции

Во всех вышестоящих экспериментах мы создавали RDD, состоящие из элементов базовых типов - числовых, срок и символов. На самом деле RDD не имеют как таковых органичений на тип элементов. Ими могут выступать коллекции, кейс 
классы, кортежи и т.д.

**PairRDD** - расширенный класс функций, доступных для RDD, где элементы - это кортеж (key, value)

In [ ]:
val pairRdd: RDD[(Char, Int)] = rdd.flatMap( x => x.toLowerCase ).map( x => (x, 1))
pairRdd.take(4).foreach(println)

countByKey подсчитывает количество кортежей по каждому ключу и возвращает локальный Map

In [ ]:
val letterCount: scala.collection.Map[Char,Long] = pairRdd.countByKey

reduceByKey работает аналогично обычному reduce, но промежуточный итог накапливается по каждому ключу независимо

In [ ]:
val letterCount: RDD[(Char, Int)] = pairRdd.reduceByKey { (x,y) => x + y }
println(letterCount.take(3).mkString(" "))

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2020.57.17.jpeg">

Join позволяет соединить два RDD по ключу. Поддерживаются join, leftOuterJoin и fullOuterJoin

In [ ]:
val favouriteLetters: Vector[Char] = Vector('a', 'd', 'o')
val favLetRdd = sc.parallelize(favouriteLetters).map(x => (x,1))


val joined: RDD[(Char, (Int, Option[Int]))] = letterCount.leftOuterJoin(favLetRdd)
joined.collect.foreach { j => 
    val (letter, (leftCount, rightCount)) = j
    rightCount match {
        case Some(v) => println(s"The letter $letter is my favourite and it appears in the RDD $leftCount times")
        case None => println(s"The letter $letter is not my favourite!")
    }
}

### Выводы
- PairRDD функции - расширенный список функций, доступный для RDD, элементы которых являются кортежем (K, V)
- PairRDD позволяют соединять два RDD по ключу K

### Полезные ссылки:
- [PairRDD API Reference](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.PairRDDFunctions)

## Работа с данными

Для изучения структуры и вычислений RDD проведем анализ датасета [Airport Codes](https://datahub.io/core/airport-codes)  

Метод `sc.textFile` позволяет прочитать файл на локальной, S3 или HDFS совместимой ФС. С помощью данного метода можно читать как обычные файлы, так и директории с файлами, а также архивы с данными.

In [ ]:
val rdd = sc.textFile("/tmp/datasets/airport-codes.csv")

Выведем первые 3 строки на экран:

In [ ]:
rdd.take(3).foreach(println)

Подготовим `case class` для парсинга данных

In [ ]:
case class Airport(
    ident: String,
    `type`: String,
    name: String,
    elevationFt: String,
    continent: String,
    isoCountry: String,
    isoRegion: String,
    municipality: String,
    gpsCode: String,
    iataCode: String,
    localCode: String,
    longitude: String,
    latitude: String
)

Уберем шапку и ненужные кавычки

In [ ]:
val firstElem = rdd.first

val noHeader = rdd.filter(x => x != firstElem).map(x => x.replaceAll("\"", ""))
noHeader.first

Напишем функцию, которая преобразует `RDD[String] => RDD[Airport]`

In [ ]:
def toAirport(data: String): Airport = {
    val airportArr: Array[String] = data.split(",").map(_.trim)
    val Array(
        ident, 
        aType, 
        name, 
        elevationFt, 
        continent, 
        isoCountry, 
        isoRegion, 
        municipality, 
        gpsCode, 
        iataCode, 
        localCode, 
        longitude,
        latitude) = airportArr
    
    Airport(
        ident = ident,
        `type` = aType,
        name = name,
        elevationFt = elevationFt,
        continent = continent,
        isoCountry = isoCountry,
        isoRegion = isoRegion,
        municipality = municipality,
        gpsCode = gpsCode,
        iataCode = iataCode,
        localCode = localCode,
        longitude = longitude,
        latitude = latitude
    )
    
}

Выполним преобразование RDD

In [ ]:
val airportRdd: RDD[Airport] = noHeader.map(x => toAirport(x))

Поскольку любые трансформации являются ленивыми, отсутствие ошибок при выполнении предыдущей ячейки еще не означает, что данная функция отрабатывает корректно на всем датасете. Проверим это с помощью операции `count`:

In [ ]:
airportRdd.count

Что произошло? `count`, как и любой action, запускает вычисление всех элементов в RDD. Если посмотреть стектрейс, мы увидим причину возникновения ошибки: 

`Caused by: scala.MatchError: [Ljava.lang.String;@42ee14f9 (of class [Ljava.lang.String;)`

Это означает, что размер массива, полученного после операции `split`, меньше количества переменных, которые мы указали в данной операции:

```scala
val Array(
        ident, 
        aType, 
        name, 
        elevationFt, 
        continent, 
        isoCountry, 
        isoRegion, 
        municipality, 
        gpsCode, 
        iataCode, 
        localCode, 
        longitude,
        latitude) = airportArr
```

Изменим код функции `toAirport`, чтобы решить данную проблему. Для простоты будем считать, что если в строке недостаточно элементов, то эту строку следует выкинуть (сделать `None`)

In [ ]:
def toAirportOpt(data: String): Option[Airport] = {
    val airportArr: Array[String] = data.split(",", -1)
    
    airportArr match {
        case Array(
            ident, 
            aType, 
            name, 
            elevationFt, 
            continent, 
            isoCountry, 
            isoRegion, 
            municipality, 
            gpsCode, 
            iataCode, 
            localCode, 
            longitude,
            latitude) => {
        
                Some(
                    Airport(
                        ident = ident,
                        `type` = aType,
                        name = name,
                        elevationFt = elevationFt,
                        continent = continent,
                        isoCountry = isoCountry,
                        isoRegion = isoRegion,
                        municipality = municipality,
                        gpsCode = gpsCode,
                        iataCode = iataCode,
                        localCode = localCode,
                        longitude = longitude,
                        latitude = latitude
                        )
                    )
        }
        case _ => Option.empty[Airport]
    }
    
}

Применим новую функцию к RDD:

In [ ]:
val airportOptRdd: RDD[Option[Airport]] = noHeader.map(toAirportOpt)

Проверим корректность выполнения функции на первых трех элементах и на всем датасете:

In [ ]:
airportOptRdd.take(3).foreach(println)
airportOptRdd.count

In [ ]:
val airportRdd: RDD[Airport] = noHeader.flatMap(toAirportOpt)

In [ ]:
airportRdd.count

Добавим корректную обработку числовых типов в наш код:

In [ ]:
case class AirportTyped(
    ident: String,
    `type`: String,
    name: String,
    elevationFt: Int,
    continent: String,
    isoCountry: String,
    isoRegion: String,
    municipality: String,
    gpsCode: String,
    iataCode: String,
    localCode: String,
    longitude: Float,
    latitude: Float
)

In [ ]:
def toAirportOptTyped(data: String): Option[AirportTyped] = {
    val airportArr: Array[String] = data.split(",", -1)
    
    airportArr match {
        case Array(
            ident, 
            aType, 
            name, 
            elevationFt, 
            continent, 
            isoCountry, 
            isoRegion, 
            municipality, 
            gpsCode, 
            iataCode, 
            localCode, 
            longitude,
            latitude) => {
        
                Some(
                    AirportTyped(
                        ident = ident,
                        `type` = aType,
                        name = name,
                        elevationFt = elevationFt.toInt,
                        continent = continent,
                        isoCountry = isoCountry,
                        isoRegion = isoRegion,
                        municipality = municipality,
                        gpsCode = gpsCode,
                        iataCode = iataCode,
                        localCode = localCode,
                        longitude = longitude.toFloat,
                        latitude = latitude.toFloat
                        )
                    )
        }
        case _ => Option.empty[AirportTyped]
    }
    
}

In [ ]:
val airportRddTyped: RDD[AirportTyped] = noHeader.flatMap(toAirportOptTyped)
airportRddTyped.count

Теперь у нас новая ошибка:

`java.lang.NumberFormatException: For input string: ""`

Она возникает, когда мы пытаемся превратить пустую строку в Int с помощью метода `.toInt`
Для решения этой задачи мы можем использовать монаду `Try[T]`

In [ ]:
import scala.util.Try

case class AirportSafe(
    ident: String,
    `type`: String,
    name: String,
    elevationFt: Option[Int],
    continent: String,
    isoCountry: String,
    isoRegion: String,
    municipality: String,
    gpsCode: String,
    iataCode: String,
    localCode: String,
    longitude: Option[Float],
    latitude: Option[Float]
)

def toAirportOptSafe(data: String): Option[AirportSafe] = {
    val airportArr: Array[String] = data.split(",", -1)
    
    airportArr match {
        case Array(
            ident, 
            aType, 
            name, 
            elevationFt, 
            continent, 
            isoCountry, 
            isoRegion, 
            municipality, 
            gpsCode, 
            iataCode, 
            localCode, 
            longitude,
            latitude) => {
        
                Some(
                    AirportSafe(
                        ident = ident,
                        `type` = aType,
                        name = name,
                        elevationFt = Try(elevationFt.toInt).toOption,
                        continent = continent,
                        isoCountry = isoCountry,
                        isoRegion = isoRegion,
                        municipality = municipality,
                        gpsCode = gpsCode,
                        iataCode = iataCode,
                        localCode = localCode,
                        longitude = Try(longitude.toFloat).toOption,
                        latitude = Try(latitude.toFloat).toOption
                        )
                    )
        }
        case _ => Option.empty[AirportSafe]
    }
    
}

Применим данную функцию к нашему датасету:

In [ ]:
val airportSafeRdd: RDD[Option[AirportSafe]] = noHeader.map(toAirportOptSafe)

Проверим ее применимость:

In [ ]:
airportSafeRdd.take(3).foreach(println)
airportSafeRdd.count

`Option[T]` - это удобная монада, которая позволяет работать с отсутствующими данными, избегая исключительных ситуаций и обработки `null`. Одним из ее преимуществ является то, что ее можно рассматривать как коллекцию, что позволяет применить к `RDD[Option[T]]` метод `flatMap`, который вернет RDD[T], убрав все `None` из нашего датасета:

In [ ]:
val airportFinal: RDD[AirportSafe] = noHeader.flatMap(toAirportOptSafe)
airportFinal.take(3).foreach(println)

Получим коллекцию, содержащую максимальную высота аэропорта с разбивкой по странам. Для этого первым шагом получим `PairRDD`: `RDD[(K,V)]`, где `K` - это страна, а `V` - высота

In [ ]:
val pairAirport = airportFinal.map(x => (x.isoCountry, x.elevationFt))
pairAirport.first

Поскольку мы не можем напрямую сравнивать два объекта `Option[T]`, то нам необходимо получить `T`. Будем считать, что аэропорты, где атрибут `elevationFt` принимает значение `None`, необходимо поместить в конец нашего списка. Для этого применим функцию:

In [ ]:
val fixedElevation: RDD[(String, Int)] = pairAirport.map {
    case (k, Some(v)) => (k, v)
    case (k, None) => (k, Int.MinValue)
}
fixedElevation.first

Теперь нам необходимо применить функцию reduceByKey и получить нужный результат:

In [ ]:
import scala.math.max

val result = fixedElevation.reduceByKey { (x, y) => Math.max(x,y) }.collect.sortBy( x => -x._2 )
result.take(10).foreach(println)

### Выводы
- RDD API - это низкоуровневый API, который позволяет применять любые функции к распределенным данным
- При использовании RDD API обработка всех исключительных ситуаций лежит на плечах разработчика

После завершения работы не забывайте останавливать `SparkSession`, чтобы освободить ресурсы кластера!

In [ ]:
spark.stop